In [10]:
import requests
import pandas as pd

# Fetch groups from Flask API
url = 'http://localhost:3000/api/memorization-group/all'  # Replace with your actual endpoint

# Function to fetch group data
def fetch_group_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("success"):
            return data['allGroupDetails']
        else:
            print("Error:", data.get("message"))
            return []
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return []

# Process and prepare group data
def process_group_data(groups):
    all_combined_info = []
    for group in groups:
        combined_info = {
            'ID': group['id'],
            'Group Name': group['group_name'],
            'Group Description': group['group_description'],
            'Capacity': group['capacity'],
            'Start Time': group['start_time'],
            'End Time': group['end_time'],
            'Group Status': group['GroupStatus']['status_name_en'],
            'Group Goal': group['GroupGoal']['group_goal_eng'],
            'Teaching Method': group.get('TeachingMethod', {}).get('methodNameEnglish', 'N/A'),
            'Supervisor': group.get('Supervisor', {}).get('fullName', 'N/A'),
            'Days': ', '.join([day['name_en'] for day in group['Days']]),
            'Languages': ', '.join([lang['name_en'] for lang in group['Languages']]),
            'Juzas': ', '.join([juza['Juza']['arabic_part'] for juza in group.get('juzas', [])])
        }
        all_combined_info.append(combined_info)
    return pd.DataFrame(all_combined_info)

# Recommendation logic
def recommend_groups(df, user_memorized_juzas, preferred_language, available_days):
    matching_groups = []

    for _, group in df.iterrows():
        # Match user's memorized Ajzaa with group's Juzas
        group_juzas = group['Juzas'].split(', ')
        if user_memorized_juzas <= len(group_juzas):
            # Match preferred language
            group_languages = [lang.strip().lower() for lang in group['Languages'].split(',')]
            if preferred_language.lower() in group_languages:
                # Match available days
                group_days = [day.strip().lower() for day in group['Days'].split(',')]
                if any(day.lower() in group_days for day in available_days):
                    matching_groups.append(group)

    return pd.DataFrame(matching_groups)

# Main execution
if __name__ == "__main__":
    # Fetch and process data
    groups = fetch_group_data(url)
    if groups:
        group_df = process_group_data(groups)

        # Get user input
        user_memorized_juzas = int(input("Enter the number of Ajzaa you have memorized: "))
        preferred_language = input("Enter your preferred language: ").strip()
        available_days = input("Enter your available days (comma-separated, e.g., Monday, Wednesday): ").strip().split(',')

        # Recommend groups
        recommended_groups = recommend_groups(group_df, user_memorized_juzas, preferred_language, available_days)

        # Display recommendations
        if not recommended_groups.empty:
            print("\nRecommended Memorization Groups:")
            display(recommended_groups[['Group Name', 'Group Description', 'Juzas', 'Languages', 'Days']])
        else:
            print("\nNo suitable groups found based on your inputs.")
    else:
        print("No group data available to process.")



Recommended Memorization Groups:


,Group Name,Group Description,Juzas,Languages,Days
0,مجموعة عمر بن الخطاب,مجموعة مخصصة لحفظ القرآن الكريم.,,arabic,"Sunday, Monday"
2,مجموعة علي بن أبي طالب - مستوى مبتدئ,مجموعة مخصصة لحفظ القرآن الكريم لجميع المستويات.,,arabic,"Sunday, Monday"
4,مجموعة خالد بن الوليد,مجموعة مخصصة لمراجعة القرآن الكريم.,,arabic,"Sunday, Monday"
6,مجموعة عبد الله بن مسعود - مراجعة,مجموعة مخصصة لمراجعة القرآن الكريم.,,arabic,"Sunday, Monday"
8,مجموعة طلحة بن عبيد الله - مراجعة,مجموعة مخصصة لمراجعة القرآن الكريم.,,arabic,"Sunday, Monday"
10,مجموعة أبو عبيدة بن الجراح - مراجعة,مجموعة مخصصة لمراجعة القرآن الكريم.,,arabic,"Sunday, Monday"
12,مجموعة عبد الله بن احمد - مراجعة,مجموعة مخصصة لمراجعة القرآن الكريم.,,arabic,"Sunday, Monday"
